In [1]:
!pip install networkx[default]
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
!git clone -b data https://github.com/tu-leminh/Community-detection.git

fatal: destination path 'Community-detection' already exists and is not an empty directory.


In [3]:
import networkx as nx
from networkx.algorithms import community
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession .builder .appName("Task 2") .getOrCreate()

In [4]:
df = spark.read.csv("Community-detection/ub_sample_data.csv",header=True)

In [5]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[user_id: string, business_id: string]>

In [6]:
df.show(3,truncate=False)

+----------------------+----------------------+
|user_id               |business_id           |
+----------------------+----------------------+
|39FT2Ui8KUXwmUt6hnwy-g|RJSFI7mxGnkIIKiJCufLkg|
|39FT2Ui8KUXwmUt6hnwy-g|fThrN4tfupIGetkrz18JOg|
|39FT2Ui8KUXwmUt6hnwy-g|mvLdgkwBzqllHWHwS-ZZgQ|
+----------------------+----------------------+
only showing top 3 rows



In [7]:
df=df.select(F.col("user_id").alias("x"),F.col('business_id')).join(df.select(F.col("user_id").alias("y"),F.col('business_id')),on="business_id",)
df.show(3,truncate=False)

+----------------------+----------------------+----------------------+
|business_id           |x                     |y                     |
+----------------------+----------------------+----------------------+
|RJSFI7mxGnkIIKiJCufLkg|39FT2Ui8KUXwmUt6hnwy-g|Et6mwzOORak5K6kJ-yw4Xw|
|RJSFI7mxGnkIIKiJCufLkg|39FT2Ui8KUXwmUt6hnwy-g|jfaz1aQqnkbtzQfQyCE8IA|
|RJSFI7mxGnkIIKiJCufLkg|39FT2Ui8KUXwmUt6hnwy-g|a0PH_2zKVQExMP5w-OJQKw|
+----------------------+----------------------+----------------------+
only showing top 3 rows



In [8]:
df=df.groupBy(["x","y"]).count()
df.show(3)

+--------------------+--------------------+-----+
|                   x|                   y|count|
+--------------------+--------------------+-----+
|39FT2Ui8KUXwmUt6h...|WPcEBAbrPZxvCGlvH...|    1|
|39FT2Ui8KUXwmUt6h...|7G8w2SnaC-qDVQ7_G...|    1|
|39FT2Ui8KUXwmUt6h...|fYJf6G30hZlhVxWTO...|    2|
+--------------------+--------------------+-----+
only showing top 3 rows



In [9]:
df=df.filter("count >= 7").select("x","y").filter("x<y")
df.count()

498

In [10]:
G = nx.Graph()
for row in df.collect():
    G.add_node(row["x"])
    G.add_node(row["y"])
    G.add_edge(row["x"], row["y"])
G=G.to_undirected()
print(len(G.edges),len(G.nodes))

498 222


In [11]:
betweenness = nx.edge_betweenness_centrality(G)
t=[(a,b) for a,b in betweenness]
sorted([[-betweenness[a],a] for a in t])

[[-0.17259793730381964, ('l-1cva9rA8_ugLrtSdKAqA', 'cyuDrrG5eEK-TZI867MUPA')],
 [-0.16250814486108608, ('DKolrsBSwMTpTJL22dqJRQ', '1st2ltGKJ00ZcRsev-Ieew')],
 [-0.15722962781786312, ('HLY9oDcVBH9D25lU4X_V5Q', '1st2ltGKJ00ZcRsev-Ieew')],
 [-0.15180791651379885, ('Hv_q_ZnSIoZwdcoH0CyV2Q', '1st2ltGKJ00ZcRsev-Ieew')],
 [-0.14958675546910835, ('JM0GL6Dx4EuZ1mprLk5Gyg', '1st2ltGKJ00ZcRsev-Ieew')],
 [-0.1359504300680771, ('l-1cva9rA8_ugLrtSdKAqA', 'HLY9oDcVBH9D25lU4X_V5Q')],
 [-0.13407525172231055, ('Hv_q_ZnSIoZwdcoH0CyV2Q', 'l-1cva9rA8_ugLrtSdKAqA')],
 [-0.09035984963113657, ('0FVcoJko1kfZCrJRfssfIA', 'DKolrsBSwMTpTJL22dqJRQ')],
 [-0.08026578614813909, ('a48HhwcmjFLApZhiax41IA', 'o-t-i7nbT5N_cmkCXs5oDQ')],
 [-0.07737148913619502, ('A-U-K9z9oraMH7eBZW1dOA', 'l-1cva9rA8_ugLrtSdKAqA')],
 [-0.07590395825689943, ('cyuDrrG5eEK-TZI867MUPA', 'o-t-i7nbT5N_cmkCXs5oDQ')],
 [-0.05710657613747684, ('JM0GL6Dx4EuZ1mprLk5Gyg', '39FT2Ui8KUXwmUt6hnwy-g')],
 [-0.05332495704864174, ('JM0GL6Dx4EuZ1mprLk5Gyg', 'K

In [12]:
#https://networkx.guide/algorithms/community-detection/girvan-newman/
import networkx as nx
from networkx.algorithms.community.centrality import girvan_newman
import networkx.algorithms.community as nx_comm

max_modularity=-100
communities = girvan_newman(G)
for i in communities:
    node_groups = []
    for com in i:
        node_groups.append(com)
    modularity=nx_comm.modularity(G, node_groups)
    if (modularity>max_modularity):
        group=node_groups
        max_modularity=modularity
        
print(group)
print(sorted([len(i) for i in group]))
print(len(group))
print(max_modularity)

[{'S1cjSFKcS5NVc3o1MkfpwA', 'FyQrUamokaPLDrBxGmzPnA', 'SX_SMrddkDU5dySbsZMu9A', 'k24kSTpZHUdEd-QYXLy3fQ', 'A-U-K9z9oraMH7eBZW1dOA', 'tcWnoX_IfuDmlDl6o6y3_g', 'LgFDWZTLi1w9OGi5BtKORg', 'Nf_Jw_W_CwOz5WJ7ApSMxg', 'LKP0Yq9T7Ss6oiDZnVtQwQ', 'ae7zi8F0B6l_JCITh1mXDg', 'BE4fE4R3TaVn8xy4sYYjbg', 'QUYbGl1DL-9faG150MQ7zA', 'jnn504CkjtfbYIwBquWmBw', 'mm9WYrFhiNqvHCyhQKw3Mg', 'ZZvfGGLnAkSBSUduV7KN-w', 'SVC0CajvmYfH5uAq4JnGvg', 'QvLg2kxqHHahxxOlHlEIZw', 'pDNeS1nbkKS7mJmhRQJPig', 'ZXyGw3Z1DyhK1sfNtpcyYA', 'h-ajC_UHD0QAyAzySN6g2A', 'DgfsJqg_gozVgaeZ5vjllA', 'ORJnGXXkS9tQBTNyPQJF9A', '9SWtEX1k9AjRg93BAzMCpg', 'B7IvZ26ZUdL2jGbYsFVGxQ', '0gZ8E5tBWTEtGEZDuTzhzw', '4ONcRRisDZkbV1cviA7nFw', 'k58KNO8Rya-q8njKq8-uBQ', 'YA-caxALI4C-eCiSM97new', 'cyuDrrG5eEK-TZI867MUPA', 'WXlxViTwXHPBvhioljN9PQ', 'o-t-i7nbT5N_cmkCXs5oDQ', 'EI9ijI9Wh66LrVW-GmWkOg', 'Gua5GdTlTWJpovtG7Hdtyg'}, {'tL2pS5UOmN6aAOi3Z-qFGg', '7Vfy39A_totC-w70qZi0MA', 'kKTcYPz47sCDH1_ylnE4ZQ', 'R4l3ONHzGBakKKNo4TN9iQ', 'e8uzNcSC5tQMD22GNAQEQA', 'Ams0iLR